In [2]:
import seaborn as sns
import pandas as pd 
import numpy as np

In [6]:
# 1. Data Ingestion (The Population)
df = sns.load_dataset('titanic')
print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

# 2. Manual Shuffle (Simulation of Sampling)
# We set a seed to ensure reproducibility for the lesson,
# but in production, this variance happens naturally.
np.random.seed(2026) 
indices = np.random.permutation(df.index)

Total Population: 891
Population Survival Rate: 0.3838


In [10]:
# 3. Cut the deck (80/20 Split)
split_point = int(len(df) * .8)

# Slicing the shuffled indices
train_idx = indices[:split_point]
test_idx = indices[split_point:]

# Creating the subsets
train_set = df.iloc[train_idx]
test_set = df.iloc[test_idx]

# 4. Bias Check (The Delta)
train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate:  {test_surv:.4f}")
print(f"Sampling Bias (Delta): {delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate:  0.4246
Sampling Bias (Delta): 0.0510


In [13]:
from sklearn.model_selection import train_test_split

# Stratify by 'pclass' ensures the distribution of classes is identical
X_train, X_test = train_test_split(df, test_size = .2, random_state = 2026, stratify = df['pclass'])

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


In [14]:
from scipy.stats import chisquare

# 1. Define observed and expected
observed = [450, 550]
expected = [500, 500]

# 2. Calculate Chi-Square statistic and p-value
chi2, p_value = chisquare(observed, f_exp=expected)

print(f"Chi-Square Statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4f}")

# 3. Conclusion
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.")
else:
    print("Variance is within natural limits.")

Chi-Square Statistic: 10.0000
P-value: 0.0016
CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.


#Why 550/450 isn't just "bad luck":


With a fair 50/50 split of 1000 users, random variance would typically produce splits very close to 500/500 — maybe 510/490 or 515/485. A 550/450 split is a 100-user deviation. The chi-square test quantifies this: the p-value will be very small (well below 0.01), meaning there's less than a 1% chance this imbalance happened by chance alone. That points to a systematic problem — like a misconfigured load balancer, a bug in the assignment logic, or bot traffic disproportionately hitting one group — not natural randomness.